## Imports

In [1]:
import altair as alt
from pylab import rcParams
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC

## Reading Data

In [2]:
df = pd.read_csv("data/crimedata_csv_AllNeighbourhoods_AllYears.csv", encoding="utf-8")
df_2023 = df[df["YEAR"] == 2023]
df_2023.head()

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
25,Break and Enter Commercial,2023,9,14,3,32,10XX ALBERNI ST,West End,491065.2962,5.459130e+06
102,Break and Enter Commercial,2023,4,1,4,7,10XX BEACH AVE,West End,490197.8719,5.458239e+06
135,Break and Enter Commercial,2023,4,3,0,50,10XX BEACH AVE,Central Business District,490249.2307,5.458167e+06
136,Break and Enter Commercial,2023,5,11,18,0,10XX BEACH AVE,Central Business District,490249.2307,5.458167e+06
185,Break and Enter Commercial,2023,8,9,4,31,10XX BEACH AVE,Central Business District,490268.4320,5.458143e+06


In [3]:
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31352 entries, 25 to 879855
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TYPE           31352 non-null  object 
 1   YEAR           31352 non-null  int64  
 2   MONTH          31352 non-null  int64  
 3   DAY            31352 non-null  int64  
 4   HOUR           31352 non-null  int64  
 5   MINUTE         31352 non-null  int64  
 6   HUNDRED_BLOCK  31352 non-null  object 
 7   NEIGHBOURHOOD  31347 non-null  object 
 8   X              31349 non-null  float64
 9   Y              31349 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 2.6+ MB


In [4]:
df_2023.describe().T

,count,mean,std,min,25%,50%,75%,max
YEAR,31352.0,2.023000e+03,0.000000e+00,2023.0,2.023000e+03,2.023000e+03,2.023000e+03,2023.000
MONTH,31352.0,5.736923e+00,2.883964e+00,1.0,3.000000e+00,6.000000e+00,8.000000e+00,11.000
DAY,31352.0,1.517310e+01,8.779039e+00,1.0,8.000000e+00,1.500000e+01,2.300000e+01,31.000
HOUR,31352.0,1.170930e+01,7.502034e+00,0.0,5.000000e+00,1.300000e+01,1.800000e+01,23.000
MINUTE,31352.0,1.792099e+01,1.891126e+01,0.0,0.000000e+00,1.300000e+01,3.000000e+01,59.000
X,31349.0,4.392056e+05,1.528544e+05,0.0,4.904529e+05,4.916470e+05,4.930989e+05,498325.516
Y,31349.0,4.867558e+06,1.693831e+06,0.0,5.454246e+06,5.457363e+06,5.458742e+06,5462265.000


In [5]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

missing_zero_values_table(df_2023)

Your selected dataframe has 10 columns and 31352 Rows.
There are 3 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
NEIGHBOURHOOD,0,5,0.0,5,0.0,object
X,3386,3,0.0,3389,10.8,float64
Y,3386,3,0.0,3389,10.8,float64


In [6]:
df_2023["TYPE"].value_counts()

TYPE
Other Theft                                               10860
Theft from Vehicle                                         6433
Mischief                                                   5461
Offence Against a Person                                   3373
Break and Enter Commercial                                 1563
Break and Enter Residential/Other                          1064
Theft of Bicycle                                            979
Vehicle Collision or Pedestrian Struck (with Injury)        933
Theft of Vehicle                                            658
Vehicle Collision or Pedestrian Struck (with Fatality)       15
Homicide                                                     13
Name: count, dtype: int64

In [7]:
df_2023["NEIGHBOURHOOD"].value_counts()

NEIGHBOURHOOD
Central Business District    10463
West End                      2430
Strathcona                    2415
Renfrew-Collingwood           2224
Mount Pleasant                1949
Fairview                      1563
Grandview-Woodland            1460
Kensington-Cedar Cottage      1351
Sunset                        1259
Kitsilano                     1054
Hastings-Sunrise               787
Marpole                        752
Riley Park                     574
Victoria-Fraserview            479
Killarney                      467
Kerrisdale                     360
Dunbar-Southlands              330
South Cambie                   288
Oakridge                       286
West Point Grey                246
Arbutus Ridge                  233
Shaughnessy                    188
Stanley Park                   155
Musqueam                        34
Name: count, dtype: int64

In [8]:
alt.data_transformers.enable('vegafusion')
numeric_cols = ["MONTH", "DAY", "HOUR", "MINUTE"]
numeric_cols_dist = alt.Chart(df_2023).mark_bar().encode(
    alt.X(alt.repeat(), type = "quantitative", bin = alt.Bin(maxbins = 30)),
    y ="count()",
).properties(
        width = 200,
        height = 150
).repeat(
    numeric_cols,
    columns = 1
)

numeric_cols_dist

alt.RepeatChart(...)

In [9]:

categ_cols_dist = alt.Chart(df_2023).mark_bar().encode(
    y = alt.X(alt.repeat(),type= "nominal").sort("x"),
    x =alt.Y("count()"),
).properties(
        width = 500,
        height = 300
).repeat(
     ["TYPE", "NEIGHBOURHOOD"],
    columns = 1
)
categ_cols_dist

alt.RepeatChart(...)

In [10]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations !")
print(get_top_abs_correlations(df_2023.select_dtypes(include=['int32','int64']), 10))

Top Absolute Correlations !
HOUR   MINUTE    0.198873
MONTH  DAY       0.026610
       MINUTE    0.016947
DAY    MINUTE    0.007530
       HOUR      0.001796
MONTH  HOUR      0.000429
YEAR   MONTH          NaN
       DAY            NaN
       HOUR           NaN
       MINUTE         NaN
dtype: float64


In [49]:
import folium
from folium.plugins import HeatMap
CBD_district=df_2023.loc[df_2023.NEIGHBOURHOOD=='Central Business District'][['X','Y']]
CBD_district.X.fillna(0, inplace = True)
CBD_district.Y.fillna(0, inplace = True) 

map_1=folium.Map(location=[49.2827, -123.1207], 
                tiles = "OpenStreetMap",
                zoom_start=11)

folium.CircleMarker([49.2727, -123.1307],
                        radius=70,
                        fill_color="#b22222",
                        popup='Other Theft',
                        color='red',
                       ).add_to(map_1)


map_2 = HeatMap(data=CBD_district, radius=20)

map_2.add_to(map_1)
map_1


In [33]:
import folium
from folium.plugins import HeatMap

# Sample data for illustration purposes
# Replace this with your actual data
B2_district_data = {
    'X': [-123.121, -123.122, -123.123],
    'Y': [49.283, 49.282, 49.281]
}

B2_district = pd.DataFrame(B2_district_data)

# Fill NaN values with 0
B2_district.X.fillna(0, inplace=True)
B2_district.Y.fillna(0, inplace=True)

# Create the base map
map_1 = folium.Map(location=[49.2827, -123.1207], 
                   tiles="OpenStreetMap",
                   zoom_start=11)

# Add a CircleMarker for illustration
folium.CircleMarker([49.2727, -123.1307],
                    radius=70,
                    fill_color="#b22222",
                    popup='TYPE',
                    color='red',
                    ).add_to(map_1)

# Create the HeatMap
heat_data = [[point['Y'], point['X']] for index, point in B2_district.iterrows()]
HeatMap(heat_data, radius=16).add_to(map_1)

# Display the map
map_1
